# Prophet

This notebook attempted to use Prophet, a library for time series forecasting. Unfortunately, this approach doesn't work in this competition, because the test dataset does not have timestamps.

In [1]:
import pandas as pd
from prophet import Prophet

# Load train data
train_df = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/train.parquet').reset_index()

# Format data for Prophet
# Prophet expects: 'ds' = datetime, 'y' = target
df_prophet = train_df[['timestamp', 'label']].rename(columns={'timestamp': 'ds', 'label': 'y'})

# Convert to datetime if needed
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'], unit='s')  # or unit='ms' depending on format

# Fit model
model = Prophet(daily_seasonality=True, weekly_seasonality=True)
model.fit(df_prophet)




10:41:29 - cmdstanpy - INFO - Chain [1] start processing
10:45:31 - cmdstanpy - INFO - Chain [1] done processing


In [2]:
train_df.head()

,timestamp,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
0,2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.121263,-0.417690,0.005399,0.125948,...,1.925423,1.847943,0.005676,0.190791,0.369691,0.377630,0.210153,0.159183,0.530636,0.562539
1,2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.302841,-0.049576,0.356667,0.481087,...,1.928569,1.849468,0.005227,0.184660,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686
2,2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,...,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505
3,2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.072944,-0.436590,-0.102483,0.017551,...,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703
4,2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.173820,-0.213489,0.096067,0.215709,...,1.927084,1.848950,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452


In [3]:
model

In [4]:
# Prepare test timestamps
test_df = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/test.parquet').reset_index()

test_df.head()

,ID,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
0,1,0.114,12.121,10.587,10.971,21.558,-0.732818,0.512331,-0.041982,-0.598260,...,1.014336,1.367567,1.584126,1.584126,1.584126,1.584125,1.567979,1.426368,2.192682,0.0
1,2,2.426,2.962,136.241,12.304,148.545,-0.337995,-0.412176,-0.259468,-0.334809,...,1.748939,1.848177,0.000131,0.000323,0.001746,0.007319,0.050925,0.142206,0.739759,0.0
2,3,1.085,2.343,23.390,57.171,80.561,0.111249,0.458221,0.466916,0.574081,...,1.704680,1.772028,0.000550,0.003597,0.009265,0.019948,0.072535,0.169672,0.807862,0.0
3,4,14.793,1.117,116.518,13.082,129.600,-0.149399,-0.640638,-0.873778,-1.026144,...,0.930946,1.037839,1.382037,1.382037,1.382037,1.382037,1.381752,1.363445,2.465509,0.0
4,5,0.033,14.178,43.800,49.836,93.636,-0.694662,0.611254,0.067671,-0.531632,...,1.080267,1.141878,0.000363,0.012718,0.137533,0.464112,1.050577,1.268567,2.499015,0.0


In [5]:
# Create dummy datetime index for test: assume 1-min intervals
test_df = test_df.reset_index(drop=True)
start_time = train_df['timestamp'].max() + pd.Timedelta(minutes=1)
test_df['timestamp'] = pd.date_range(start=start_time, periods=len(test_df), freq='T')

X_test = TimeSeries.from_dataframe(test, time_col='timestamp', value_cols=features)


/tmp/ipykernel_35/2674884027.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  test_df['timestamp'] = pd.date_range(start=start_time, periods=len(test_df), freq='T')


NameError: name 'TimeSeries' is not defined

In [ ]:
future = test_df[['ID']].rename(columns={'ID': 'ds'})
future['ds'] = pd.to_datetime(future['ds'], unit='s')  # or unit='ms'



In [ ]:
# Forecast
forecast = model.predict(future)

# Prepare submission
submission = pd.read_csv('/kaggle/input/drw-crypto-market-prediction/sample_submission.csv')
submission['label'] = forecast['yhat']
submission.to_csv('submission_prophet.csv', index=False)